In [34]:
import numpy as np
import os
import cv2
import h5py


In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
import prepare_data as pd

데이터 전처리

      - train, test 데이터 가공

In [36]:
Random_Crop = 30
Patch_size = 32
label_size = 20
conv_side = 6
scale = 2 

def prepare_data(_path):
    names = os.listdir(_path)
    names = sorted(names)
    nums = names.__len__()

    data = np.zeros((nums * Random_Crop, 1, Patch_size, Patch_size), dtype=np.double)     # data = zeros(14*30, 1, 32, 32)   이미지 단일화
    label = np.zeros((nums * Random_Crop, 1, label_size, label_size), dtype=np.double)    # label = zeros(14*30, 1, 20, 20)  이미지 단일화

    for i in range(nums):
        name = _path + names[i]
        hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
        shape = hr_img.shape

        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
        hr_img = hr_img[:, :, 0]
        
        # 저화질 이미지 생성 (행렬로만 이루어져 있음)
        # lr_img = cv2.resize(hr_img, (shape[1] / scale, shape[0] / scale))           # shape (h, w)
        # lr_img = cv2.resize(lr_img, (shape[1], shape[0]))
        
        w, h, c = shape
        new_height = int(h / scale)
        new_width = int(w / scale)
        lr_img = cv2.resize(hr_img, (new_height, new_width))           # shape (h, w)
        lr_img = cv2.resize(lr_img, (shape[1], shape[0]))

        # produce Random_Crop random coordinate to crop training img
        Points_x = np.random.randint(0, min(shape[0], shape[1]) - Patch_size, Random_Crop)        # points_x = random(0, 낮은 값 - 32, 30)
        Points_y = np.random.randint(0, min(shape[0], shape[1]) - Patch_size, Random_Crop)        # points_y = random(0, 낮은 값 - 32, 30)
        
        # 이미지 단일화
        for j in range(Random_Crop):
            lr_patch = lr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]            # lr_patch = (n번째 + 32, n번째 + 32)
            hr_patch = hr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]

            lr_patch = lr_patch.astype(float) / 255.
            hr_patch = hr_patch.astype(float) / 255.

            data[i * Random_Crop + j, 0, :, :] = lr_patch
            label[i * Random_Crop + j, 0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]
            # cv2.imshow("lr", lr_patch)
            # cv2.imshow("hr", hr_patch)
            # cv2.waitKey(0)
    return data, label

In [42]:
# train 데이터
train_y, train_x = prepare_data('./train/')
print(train_x.shape, train_y.shape)

(2400, 1, 20, 20) (2400, 1, 32, 32)


In [43]:
# test 데이터
test_y, test_x = prepare_data('./test/')
print(test_x.shape, test_y.shape)

(600, 1, 20, 20) (600, 1, 32, 32)


h5파일로 저장

In [20]:
def write_hdf5(data, labels, output_filename):
    """
    This function is used to save image data and its label(s) to hdf5 file.
    output_file.h5,contain data and label
    """

    x = data.astype(np.float32)
    y = labels.astype(np.float32)

    with h5py.File(output_filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)
        # h.create_dataset()

In [44]:
# train, test 데이터 저장
write_hdf5(train_x, train_y, 'train.h5')
write_hdf5(test_x, test_y, 'test.h5')

In [28]:
data, label = pd.read_training_data("training.h5")

모델링

In [31]:
# define model type
SRCNN = Sequential()
    
 # add model layers
SRCNN.add(Conv2D(filters=128, kernel_size=(9, 9), kernel_initializer='glorot_uniform',
                     activation='relu', padding='valid', use_bias=True, input_shape=(None, None, 1)))
SRCNN.add(Conv2D(filters=64, kernel_size=(3, 3), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', use_bias=True))
SRCNN.add(Conv2D(filters=1, kernel_size=(5,5), kernel_initializer='glorot_uniform',
                     activation='linear', padding='valid', use_bias=True))
    
# compile model
SRCNN.compile(optimizer='adam', loss='mse', metrics=['mse'])
SRCNN.fit(data, label, batch_size=128, shuffle=True, epochs=200, verbose=0)
    